# Dynamic Prompt
<img src="./assets/LC_DynamicPrompts.png" width="500">

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

# Declaring default model
default_model = "google_genai:gemini-2.5-flash"

OPENAI_API_KEY=<not set>
LANGSMITH_API_KEY=****3d99
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass


@dataclass
class RuntimeContext:
    is_employee: bool
    db: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
{table_limits}
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

## Build a Dynamic Prompt
Utilize runtime context and middleware to generate a dynamic prompt.

In [6]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt


@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    if not request.runtime.context.is_employee:
        table_limits = "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
    else:
        table_limits = ""

    return SYSTEM_PROMPT_TEMPLATE.format(table_limits=table_limits)

Include middleware in `create_agent`.

In [7]:
from langchain.agents import create_agent

agent = create_agent(
    model=default_model,
    tools=[execute_sql],
    middleware=[dynamic_system_prompt],
    context_schema=RuntimeContext,
)

In [8]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=False, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================

[{'type': 'text', 'text': 'I cannot fulfill this request as I do not have access to customer or purchase information. I can only access information about Albums, Artists, Genres, Playlists, and Tracks.', 'extras': {'signature': 'CpADAXLI2nzZrbdfP/JbkX/WeafyxYdQcjEvszETbqdUIw+XuvArMNXoPkbTodYaz1BpNFhSPk44e/5Mujz4QqOkH+RsLhBrFdIEJ+AWPrZG9v/JL6Ov8Y7cJarvKG5GvtML7C9JcnkjHSYuXPMWK6QTREJtp0XNa2BQUdOx8aLxchMOkLP2EdPe45UNAJ4paInvKZXW7WBulRyOoZVZdJszf50y6n96CBjbm++3zqAIrO99b3F8oCzAMIOHCu/0NU54nJ0jBdnnyA/ENxqxOCXMeRnhTKT+9ZBSL9hLJbUELPRTSYkHkPdNjE2/YJRHQb45mno9NY1R4nwDizCjxKGmb2j7QM/iKirxuHsMIG98TkIXRDSKbM4AJecLrkF+mGUNO9ANCgphY1Pmu5LXBbBnWT63hizG+3JTF+zoyua6DCevJHTmpqbvUi/nUa47mPz5UMRwPtwAG++w/HbiEFSgBcV0SbXZyd8wuCBrD6BS7Bpy92valwF22s7ZC7QhmcMdP0vG18Gl9VB9KfO5GcU+mA=='}}]


In [9]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=True, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (870530f1-af24-46a8-879f-e2dd3366a12f)
 Call ID: 870530f1-af24-46a8-879f-e2dd3366a12f
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table';
================================= Tool Message =================================
Name: execute_sql

[('Album',), ('Artist',), ('Customer',), ('Employee',), ('Genre',), ('Invoice',), ('InvoiceLine',), ('MediaType',), ('Playlist',), ('PlaylistTrack',), ('Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (33439718-c0ef-4e7f-b90c-7cfa5c7c98fc)
 Call ID: 33439718-c0ef-4e7f-b90c-7cfa5c7c98fc
  Args:
    query: SELECT CustomerId FROM Customer WHERE FirstName = 'Frank' AND LastName = 'Harris';
================================= Tool Message 